In [1]:
# -------------------------------------------------
# 0. Imports & GPU
# -------------------------------------------------
import os, random, warnings, numpy as np, matplotlib.pyplot as plt
from pathlib import Path
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from PIL import Image
import cv2
import json
from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore')
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)

print("TF:", tf.__version__)
print("GPU:", tf.config.list_physical_devices('GPU'))

# -------------------------------------------------
# 1. GLOBAL SETTINGS (200 classes only)
# -------------------------------------------------
DATA_ROOT     = Path(r"C:\Users\TIK03\Documents\GitHub\DIT5411-HoYiTik\Assgnment\data\characters")
NOTEBOOK_ROOT = Path(r"C:\Users\TIK03\Documents\GitHub\DIT5411-HoYiTik\Assgnment")

IMG_SIZE      = (64, 64)
SAMPLE_CHARS  = 200
TARGET_PER_CLASS = 200
EPOCHS        = 12
BATCH_SIZE    = 32

# -------------------------------------------------
# 2. Load folders
# -------------------------------------------------
char_folders = sorted([p for p in DATA_ROOT.iterdir() if p.is_dir()])[:SAMPLE_CHARS]
print(f"Selected {len(char_folders)} character folders (first 10):")
print([p.name for p in char_folders[:10]])

# -------------------------------------------------
# 3. Loader
# -------------------------------------------------
def safe_pil_read(p):
    try:
        return np.array(Image.open(p).convert('L'))
    except Exception as e:
        print(f"  [WARN] {p.name}: {e}")
        return None

def load_class(folder: Path):
    files = sorted(folder.glob("*.png"))
    imgs = []
    for p in files:
        arr = safe_pil_read(p)
        if arr is not None:
            imgs.append(cv2.resize(arr, IMG_SIZE, interpolation=cv2.INTER_AREA))
    return np.array(imgs, dtype=np.uint8) if imgs else None

# -------------------------------------------------
# 4. Load + CONTINUOUS LABELS
# -------------------------------------------------
print(f"\nLoading {len(char_folders)} classes …")
train_X_list, train_y_list = [], []
test_X_list , test_y_list  = [], []
label_to_char = {}
char_to_label = {}
valid_class_idx = 0

for folder in char_folders:
    imgs = load_class(folder)
    if imgs is None or len(imgs) < 40:
        print(f"  [SKIP] {folder.name} – <40 images")
        continue

    label_to_char[valid_class_idx] = folder.name
    char_to_label[folder.name] = valid_class_idx

    n_train = 40
    train_X_list.append(imgs[:n_train])
    train_y_list.extend([valid_class_idx] * n_train)

    if len(imgs) > n_train:
        test_X_list.append(imgs[n_train:])
        test_y_list.extend([valid_class_idx] * (len(imgs) - n_train))

    valid_class_idx += 1

if not train_X_list:
    raise RuntimeError("No valid classes!")

train_X = np.concatenate(train_X_list)[..., np.newaxis] / 255.0
train_y = np.array(train_y_list, dtype=np.int32)
test_X = np.concatenate(test_X_list)[..., np.newaxis] / 255.0 if test_X_list else None
test_y = np.array(test_y_list, dtype=np.int32) if test_y_list else None

n_classes = len(label_to_char)
print(f"Valid classes: {n_classes}")
print(f"Train: {train_X.shape}  Test: {test_X.shape if test_X is not None else 'None'}")

# -------------------------------------------------
# 5. Augmentation
# -------------------------------------------------
def random_transform(img):
    img = (img.squeeze() * 255).astype(np.uint8)
    h, w = 64, 64
    angle = random.choice([-12, -8, -4, 0, 4, 8, 12])
    M = cv2.getRotationMatrix2D((w//2, h//2), angle, 1.0)
    img = cv2.warpAffine(img, M, (w, h))
    shear = random.uniform(0.08, 0.22)
    M = np.float32([[1, shear, 0], [0, 1, 0]]) if random.random() > 0.5 else np.float32([[1, 0, 0], [shear, 1, 0]])
    img = cv2.warpAffine(img, M, (w, h))
    scale = random.uniform(0.82, 1.18)
    nw, nh = int(w * scale), int(h * scale)
    img = cv2.resize(img, (nw, nh), interpolation=cv2.INTER_AREA)
    top = bottom = left = right = 0
    if nh < h: pad = (h - nh) // 2; top, bottom = pad, h - nh - pad
    elif nh > h: crop = (nh - h) // 2; img = img[crop:crop + h, :]
    if nw < w: pad = (w - nw) // 2; left, right = pad, w - nw - pad
    elif nw > w: crop = (nw - w) // 2; img = img[:, crop:crop + w]
    if any((top, bottom, left, right)):
        img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=255)
    return (img / 255.0)[..., np.newaxis]

def augment_to_target(class_imgs, target=200):
    aug = []
    while len(aug) < target:
        for img in class_imgs:
            aug.append(random_transform(img))
            if len(aug) >= target: break
    return np.array(aug[:target])

print("Augmenting to 200 samples per class …")
aug_X, aug_y = [], []
for label in range(n_classes):
    idxs = np.where(train_y == label)[0]
    class_imgs = train_X[idxs]
    aug = augment_to_target(class_imgs, target=TARGET_PER_CLASS)
    aug_X.append(aug)
    aug_y.extend([label] * len(aug))

train_X = np.concatenate(aug_X)
train_y = np.array(aug_y, dtype=np.int32)
print(f"Final train: {train_X.shape}  ({train_X.shape[0]//n_classes} per class)")

# -------------------------------------------------
# 6. Train/Val Split
# -------------------------------------------------
train_X_final, val_X, train_y_final, val_y = train_test_split(
    train_X, train_y, test_size=0.2, random_state=42, stratify=train_y
)

# -------------------------------------------------
# 7. Model Builder
# -------------------------------------------------
def build_cnn(arch, input_shape=(64,64,1), n_classes=None):
    if arch == 'cnn_res':
        inputs = layers.Input(shape=input_shape)
        x = layers.Conv2D(32, 3, activation='relu')(inputs)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling2D(2)(x)
        shortcut = x
        x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Conv2D(64, 3, padding='same', activation=None)(x)
        x = layers.BatchNormalization()(x)
        shortcut = layers.Conv2D(64, 1, padding='same')(shortcut)
        x = layers.Add()([x, shortcut])
        x = layers.Activation('relu')(x)
        x = layers.MaxPooling2D(2)(x)
        x = layers.Conv2D(128, 3, padding='same', activation='relu')(x)
        x = layers.MaxPooling2D(2)(x)
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Dense(256, activation='relu')(x)
        x = layers.Dropout(0.5)(x)
        outputs = layers.Dense(n_classes, activation='softmax')(x)
        model = models.Model(inputs, outputs, name='cnn_res')
    else:
        model = models.Sequential(name=arch)
        model.add(layers.Conv2D(32, 3, activation='relu', input_shape=input_shape))
        model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling2D(2))
        if arch == 'cnn2':
            model.add(layers.Conv2D(64, 3, activation='relu'))
            model.add(layers.MaxPooling2D(2))
            model.add(layers.Flatten())
            model.add(layers.Dense(128, activation='relu'))
            model.add(layers.Dropout(0.4))
        elif arch == 'cnn3':
            model.add(layers.Conv2D(64, 3, activation='relu'))
            model.add(layers.MaxPooling2D(2))
            model.add(layers.Conv2D(128, 3, activation='relu'))
            model.add(layers.MaxPooling2D(2))
            model.add(layers.Flatten())
            model.add(layers.Dense(256, activation='relu'))
            model.add(layers.Dropout(0.5))
        model.add(layers.Dense(n_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# -------------------------------------------------
# 8. TRAIN LOOP (FIXED CHECKPOINT)
# -------------------------------------------------
results = {}
histories = {}
early_cb = callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True, verbose=1)

for arch in ['cnn2', 'cnn3', 'cnn_res']:
    print(f"\n{'='*20} TRAINING {arch.upper()} {'='*20}")
    
    # FIXED: Use f-string with current `arch`
    checkpoint_cb = callbacks.ModelCheckpoint(
        filepath=str(NOTEBOOK_ROOT / f"{arch}_best.keras"),
        save_best_only=True,
        monitor='val_accuracy',
        mode='max',
        verbose=1
    )

    model = build_cnn(arch, n_classes=n_classes)
    hist = model.fit(
        train_X_final, train_y_final,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_data=(val_X, val_y),
        callbacks=[checkpoint_cb, early_cb],
        verbose=1
    )
    histories[arch] = hist.history

    # Evaluate on real test set
    if test_X is not None:
        test_loss, test_acc = model.evaluate(test_X, test_y, verbose=0)
        print(f"{arch} → Test Accuracy: {test_acc:.4%}")
        results[arch] = test_acc
    else:
        acc = hist.history['val_accuracy'][-1]
        print(f"{arch} → Val Accuracy: {acc:.4%}")
        results[arch] = acc

    model.save(str(NOTEBOOK_ROOT / f"{arch}_final.keras"))

# -------------------------------------------------
# 9. Summary
# -------------------------------------------------
print("\n" + "="*60)
print("FINAL TEST ACCURACY (real test images)")
print("="*60)
for arch, acc in results.items():
    print(f"{arch:8} : {acc:.4%}")
print("="*60)

# -------------------------------------------------
# 10. Save label map
# -------------------------------------------------
with open(NOTEBOOK_ROOT / "label_to_char.json", "w", encoding="utf-8") as f:
    json.dump(label_to_char, f, ensure_ascii=False, indent=2)
print(f"Label map saved → {NOTEBOOK_ROOT / 'label_to_char.json'}")

TF: 2.20.0
GPU: []
Selected 200 character folders (first 10):
['1', '10', '100', '1000', '10000', '10001', '10002', '10003', '10004', '10005']

Loading 200 classes …
  [SKIP] 10020 – <40 images
Valid classes: 199
Train: (7960, 64, 64, 1)  Test: (3428, 64, 64, 1)
Augmenting to 200 samples per class …
Final train: (39800, 64, 64, 1)  (200 per class)

==================== TRAINING CNN2 ====================
Epoch 1/12
995/995 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.0044 - loss: 5.3124
Epoch 1: val_accuracy improved from None to 0.00503, saving model to C:\Users\TIK03\Documents\GitHub\DIT5411-HoYiTik\Assgnment\cnn2_best.keras
995/995 ━━━━━━━━━━━━━━━━━━━━ 19s 18ms/step - accuracy: 0.0036 - loss: 5.2977 - val_accuracy: 0.0050 - val_loss: 5.2933
Epoch 2/12
993/995 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.0053 - loss: 5.2960
Epoch 2: val_accuracy did not improve from 0.00503
995/995 ━━━━━━━━━━━━━━━━━━━━ 18s 18ms/step - accuracy: 0.0039 - loss: 5.2950 - val_accuracy: 0.0050 - val_